In [1]:
import re
from collections import Counter

import pandas as pd
import ray
import multiprocessing

#multiple outputs per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
df = pd.read_csv('data/pwAnalysis.csv',usecols=['A'])

In [5]:
df.head()

,A
0,07606374520
1,piontekendre
2,rambo144
3,primoz123
4,sal1387


In [3]:
ray.init()  #kick off multi-threading

@ray.remote
def mostCommonStrings(charLen,s):
    cnt = Counter()
    t = len(s) - (charLen - 1)
    
    for i in range(t):
        
        # a sequence of characters being considered
        char = s[i:i+charLen]                
        
        # let's only look at alpha strings
        allAlpha = True
        for j in range(charLen):
            if not char[j].isalpha():
                allAlpha = False
                break
            
        if allAlpha:
            numFound = len(re.findall(char,s))
            if numFound>1:
                cnt[char] = numFound
    
    return dict(cnt)

Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:32096 to respond...
Waiting for redis server at 127.0.0.1:16366 to respond...
Starting local scheduler with the following resources: {'CPU': 4, 'GPU': 0}.

View the web UI at http://localhost:8890/notebooks/ray_ui63279.ipynb?token=b750bd60c0a183a665601a0bb4e3033f3d302b0ecdd1cafa



{'local_scheduler_socket_names': ['/tmp/scheduler51221488'],
 'node_ip_address': '127.0.0.1',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store52808779', manager_name='/tmp/plasma_manager73697470', manager_port=64833)],
 'redis_address': '127.0.0.1:32096',
 'webui_url': 'http://localhost:8890/notebooks/ray_ui63279.ipynb?token=b750bd60c0a183a665601a0bb4e3033f3d302b0ecdd1cafa'}

In [4]:
threads = multiprocessing.cpu_count()
print(df.size)
batchSize = df.size / threads
batchSize = 100000 / threads
print('threads, batchSize = {}, {}'.format(threads,batchSize))

2151220
threads, batchSize = 4, 25000.0


In [5]:
%%time

for charLen in range(4,9):   # look at strings of len 4 up to 8
    cnt = Counter()
    for i in range(threads):
        s = df[int(i * batchSize) : int(i * batchSize + batchSize) ].A.str.cat()
        cnt += ray.get(mostCommonStrings.remote(charLen,s)) 
    print("most common {} digit strings are {}".format(charLen,cnt.most_common(5)))

most common 4 digit strings are [('love', 574), ('wang', 286), ('hang', 285), ('chen', 274), ('iang', 256)]
most common 5 digit strings are [('zhang', 182), ('admin', 180), ('ilove', 162), ('angel', 89), ('huang', 87)]
most common 6 digit strings are [('ssword', 64), ('asswor', 62), ('passwo', 59), ('master', 55), ('woaini', 52)]
most common 7 digit strings are [('assword', 60), ('passwor', 59), ('forever', 36), ('dearboo', 33), ('earbook', 33)]
most common 8 digit strings are [('password', 57), ('dearbook', 33), ('iloveyou', 19), ('computer', 12), ('kikugala', 10)]
CPU times: user 218 ms, sys: 15.4 ms, total: 233 ms
Wall time: 3min 59s


In [6]:
# df2 = df[0: int(3*batchSize + batchSize)]
# df2[df2.A.str.contains('love',na=False)].size